# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [45]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

import re
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jiaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jiaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jiaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jiaxi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterResponse', engine)
X = df['message'].values
Y = df.iloc[:, 4:].values
categories = df.columns[4:]

In [14]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

### 2. Write a tokenization function to process your text data

In [54]:
def tokenize(text):
    # normalization
    text = text.lower() # capitalization removal
    text = re.sub(r"[^a-z0-9]", " ", text) # punctuation removal
    
    # tokenization
    tokens = word_tokenize(text)
    # stop word removal
#     tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()
    # lemmatization
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)
    return clean_tokens

In [36]:
tokenize(X[1])

['hurricane']

In [37]:
CountVectorizer().fit_transform(tokenize(X[1])).toarray()

array([[1]], dtype=int64)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [55]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [56]:
# Split data. 25% test data by default
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# train pipeline
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000002920AB753A0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [57]:
# prediction
y_pred = pipeline.predict(X_test)

In [58]:
# evaluation
# for i in range(36):
#     y_i_pred = y_pred[:, i]
#     y_i_test = y_test[:, i]
#     print(classification_report(y_i_test, y_i_pred, target_names = categories[i]))
print(classification_report(y_test, y_pred, target_names = categories))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4926
               request       0.89      0.46      0.60      1137
                 offer       0.00      0.00      0.00        29
           aid_related       0.78      0.63      0.70      2728
          medical_help       0.68      0.07      0.12       527
      medical_products       0.74      0.06      0.11       334
     search_and_rescue       1.00      0.05      0.10       186
              security       0.20      0.01      0.02        99
              military       0.60      0.03      0.06       195
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.33      0.48       428
                  food       0.86      0.55      0.67       765
               shelter       0.85      0.30      0.44       576
              clothing       0.50      0.03      0.06        96
                 money       0.43      

C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 6. Improve your model
Use grid search to find better parameters. 

In [34]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [60]:
parameters = {'clf__estimator__n_estimators': [50, 100],
             'clf__estimator__min_samples_split': [2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = 'f1_micro')

# find the best combination of params
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000002920AB753A0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5, 10],
                         'clf__estimator__n_estimators': [50, 100]},
             scoring='f1_micro')

In [61]:
cv.best_params_

{'clf__estimator__min_samples_split': 5, 'clf__estimator__n_estimators': 100}

In [62]:
model = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [63]:
# predict and evaluate on the test data
y_pred = cv.predict(X_test)
print(classification_report(y_test, y_pred, target_names = categories))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      4926
               request       0.89      0.48      0.63      1137
                 offer       0.00      0.00      0.00        29
           aid_related       0.77      0.65      0.71      2728
          medical_help       0.65      0.07      0.13       527
      medical_products       0.75      0.07      0.13       334
     search_and_rescue       1.00      0.04      0.07       186
              security       0.00      0.00      0.00        99
              military       0.53      0.04      0.08       195
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.36      0.51       428
                  food       0.88      0.42      0.57       765
               shelter       0.82      0.32      0.46       576
              clothing       0.50      0.03      0.06        96
                 money       0.75      

C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [66]:
# try Adaboost
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# params
parameters2 = {'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__learning_rate': [0.5, 1, 5]}

# grid search
cv2 = GridSearchCV(pipeline2, param_grid = parameters2, scoring = 'f1_micro')

In [69]:
# best model
cv2.fit(X_train, y_train)
model2 = cv2.best_estimator_
y_pred2 = model2.predict(X_test)
print(classification_report(y_test, y_pred2, target_names = categories))

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      4926
               request       0.76      0.58      0.66      1137
                 offer       0.00      0.00      0.00        29
           aid_related       0.76      0.61      0.67      2728
          medical_help       0.60      0.26      0.36       527
      medical_products       0.64      0.34      0.45       334
     search_and_rescue       0.59      0.20      0.30       186
              security       0.09      0.02      0.03        99
              military       0.62      0.31      0.41       195
           child_alone       0.00      0.00      0.00         0
                 water       0.76      0.67      0.71       428
                  food       0.81      0.70      0.75       765
               shelter       0.73      0.54      0.62       576
              clothing       0.71      0.38      0.49        96
                 money       0.57      

C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Jiaxi\Anaconda3\lib\site-packages\sklearn\metrics\_cla

### 9. Export your model as a pickle file

In [70]:
# the best classifier is RandomForest with min sample split = 5, num of estimators = 100
pickle.dump(model, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.